# CSRF攻击
## CSRF攻击概述
- CSRF（Cross Site Request Forgery,跨站域请求伪造）是一种网络的攻击方式，在2007年被列为互联网20大安全隐患之一。其他安全隐患，比如SQL脚本注入，跨站域脚本攻击等在近年来已逐渐为众人所熟知，很多网站也都针对他们进行了防御，然而，对于大多数然来说，依然不懂CSRF

## CSRF攻击原理
- 网站是通过cookie来实现登录功能的，而cookie只要存在于浏览器中，那么浏览器在访问这个cookie的服务器的时候，就会自动的携带cookie信息到服务器上去，那么这个时候就存在一个漏洞了。如果你访问了一个病毒网站，这个网站可以在网页源代码中插入js代码（js代码中含有一个iframe标签，攻击主要就是通过iframe来实现的），使用js代码给其他服务器发送请求（比如ICBC转账请求）。由于在发送请求的时候浏览器会自动的把cookie发送给对应的服务器（相当于是病毒网站盗用了你的cookie信息），此时相应的服务器无法判断出请求是伪造的，就被欺骗过去了。从而达到在用户不知情的情况下，给某个服务器发送了一个请求，实现了自己攻击的目的
- 自己的理解：
    - - 首先浏览器有一个特性：浏览器在向服务器发送请求的时候会自动发送服务器曾保存在浏览器中相应的cookie（如果曾经你访问过这个网站就会有cookie信息）。而CSRF攻击的要点就是：在你访问病毒网站的时候，病毒网站利用这个特性悄悄的使用iframe标签向某个服务器（比如银行转账页面）发送请求（此时请求中已经包含了你的浏览器上的cookie），造成服务器不知道这个请求是用户发起的还是伪造的，已到达攻击的目的。
    
## iframe相关知识
- 1.iframe可以加入嵌套入别的域名下的网页，也就是说可以发送跨域请求，比如可以在我们自己的网页中加载百度的网站
    - 示例代码：
            iframe src="http://www.baidu.com/"
            /iframe
- 2.因为iframe加载的是别的域名下的网页，根据*同源策略*,js只能操作属于本域名下的代码，因此js不能通过iframe加载来的dom元素
- 3.如果iframe的src属性为空，那么就没有同源策略的限制，这时候就可以操作iframe下面的代码了。并且，如果src为空，那么可以直接在iframe中，给任何域名都能发送请求
- 4.直接在iframe中写HTML代码，浏览器是不会加载的

- 攻击代码模板见下：
    - 主要就是黑客伪造post请求来请求目标服务器，此时要把目标服务器的'django.middleware.csrf.CsrfViewMiddleware',注释掉才能生效


## 防御CSRF攻击
- 为了防御CSRF攻击，可以在用户每次访问表单的页面的时候，在网页源代码中加一个随机的字符串叫做csrf_token，在cookie中也加入一个相同值的csrftoken字符串，以后在给服务器发送请求的时候，必须在body中以及cookie中都携带csrftoken，服务器只有检测到cookie中的csrftoken和body中的csrftoken都相同，才认为这个请求是正常的，否则就是伪造的。那么黑客就没办法伪造请求了

### csrf_token的工作原理
- 首先你第一次在向一个服务器时（get请求）请求一个页面时，服务器就会返回给你一个页面；但此时服务器还额外做了一些操作：它会在返回的页面表单（即form标签下）以及cookie中添加一个csrftoken字段。以后浏览器在发送post请求时，服务器会检查form标签下的csrftoken和cookie中的csrftoken是否一致，如果不一致，就拒绝处理。
- 那么为什么黑客在伪造post请求的时候不能伪造csrftoken字段呢；第一：csrftoken字段是服务器随机生成的，而黑客是不能更改服务器传回的cookie信息的；第二：为啥不能伪造表单中的csrftoken字段来匹配cookie的csrftoken字段，因为黑客伪造的post请求是通过js代码来实现的，在js代码中是无法通过爬虫来获取服务器返回页面中csrftoken的值的。所以无法伪造

### 1.django如何设置来防御csrf攻击
- 普通表单中通过直接加{% csrf_token %}即可：具体见下：
    - 在MIDDLEWARE中开启：'django.middleware.csrf.CsrfViewMiddleware',然后在 templete 中, 为每个 POST form 增加一个 {% csrf_token %}

#### 2.使用ajax防御csrf攻击
- 如果用ajax来处理CSRF防御，那么需要手动的在form中添加csrfmiddlewaretoken，或者是在请求头当中添加x-CSRFToken,可以从返回的cookie中提取csrf_token，再设置进去
- ajax使用添加请求头的方式来添加x-CSRFToken：ajax请求的表单不需要form标签
    - 首先在项目下建立一个static文件，里面放入js文件（比如myajax.js），同时将static注册到settings.py文件中：STATICFILES_DIRS = [os.path.join(BASE_DIR, 'static'),]
    - 在模板html文件中引入static: <% load static %>
        - 具体代码见下面

# XSS攻击
- xss(cross site script)攻击，又叫做跨站脚本攻击，原理是用户在使用具有xss漏洞的网站的时候，向这个网站提交一些恶意的脚本代码，当用户在访问这个网站的某个页面的时候，这个恶意的代码就会被执行，从而破坏网页的结构，获取用户的隐私信息

# Xss攻击场景
- 比如A网站有个一发布帖子的入口，如果用户提交数据的时候，提交了一段js代码，比如：<script>alert("hello world");</script>,然后A网站在渲染这个帖子的时候，直接把这个代码渲染了， 那么这个代码就会执行，会在这个浏览器的窗口中弹出一个模态对话框来显示helloworld，如果攻击者能成功的运行以上这么一段代码，那么他就能做很多操作来获取自己想要的信息。

# XSS攻击防御：
- 1.如果不需要显示一些富文本，那么在渲染用户的提交的数据的时候，直接进行转义就可以了。在django的模板中默认就是转义的（如果不想默认转义，加safe即可），也可以吧数据在存储到数据库之前，就转义再存储进去，以后在做渲染的时候，即使不转义也不会有安全问题
    - 在存储时转义：
            from django.template.defaultfilters import escape
            # 当中间没有Form表单验证时，直接使用decorators中的方法来修饰这些方法，表明请求时该做啥
            # from django.views.decorators.http import require_http_methods
            @require_http_methods(['POST'])
            def add_comment(request):
                content = request.POST.get('content')
                content = escape(content)
                # 在这里设置转义后，哪怕你之前设置了safe也不管用
                Comment.objects.create(content=content)
                return redirect(reverse("index"))
- 见下列代码 

- 2.如果对于用户提交上来的数据包含了一些富文本（比如：给字体上颜色，字体加粗等），那么这时候我们在渲染的时候也要以富文本的形式进行渲染，即需要使用safe过滤器将其标记为安全的，这样才能显示出富文本的样式，但是这样又会存在一个问题，如果用户提交的是攻击的脚本，则使用safe是有安全隐患的。
- 此时就可以指定某些标签是我们需要的（比如:span标签），而某些标签我们是不需要的（比如：script），那么我们在服务器处理的数据的时候，我可以将这些需要的标签保留下来，把那些不需要的标签进行转义，或者干脆移除掉，这样就可以解决我们的问题了。这个方法是可行的。包括很多线上的网站也是这样做的。在python中有一个库专门用来处理这个事情，就是bleach

# bleach库
- bleach库是用来清理包含HTML格式字符串的库，它可以指定哪些标签需要保留，哪些标签是需要过滤的，也可以指定标签上哪些属性是可以保留，哪些属性时不需要的，想要使用这个库，先进行安装
- 这个库最重要的一个方法是bleach.clean()方法
- 要先设置safe，代码见下面

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>首页</title>
</head>
<body>
<ul>
    {% for comment in comments %}
        {#当设置safe以后，意思是相信他，就不会对标签进行转义，那么用户在输入的时候就能输入脚本进行操作#}
        {#比如输入：<script>alert("hello everyone")</script>， 浏览器就会弹出helloeveryone#}
        <li>{{ comment.content|safe }}</li>
    {% endfor %}
    <form action="{% url 'add_comment' %}" method="post">
        <fieldset style="border: 0;">
             {% csrf_token %}
            <textarea name="content" id="comment_content" cols="30" rows="10"></textarea>
            <p><input type="submit" value="提交"></p>
        </fieldset>
    </form>

</ul>
</body>
</html>

In [ ]:
# bleach代码示例
import bleach
from bleach.sanitizer import ALLOWED_TAGS,ALLOWED_ATTARIBUTES

@require_http_methods(['POST'])
def message(request):
    # 从客户端中获取提交的数据
    content = request.POST.get("content")
    # 在默认的允许属性中添加img标签
    tags = ALLOWED_TAGS + ['img']
    # 在默认的允许属性中添加src属性
    attributes = {**ALLOWED_ATTRIBUTES, 'img':['src']}

    # 对提交的数据进行过滤
    cleaned_content = bleach.clean(content, tags=tags, attributes=attributes)
    # 保存数据到数据库中
    Comment.objects.create(content=cleaned_content)
    return redirect(reverse("index"))

"""
相关解释：
1.tags：表示允许哪些标签
2.attributes：表示标签中运行有哪些属性
3.ALLOWED_TAGS:这个变量是bleach默认定义的一些标签，如果不符合要求，可以对其进行增加或者删除
4.ALLOWED_ATTRIBUTES:这个变量是bleach默认定义的一些属性，如果不符合要求，可以对其进行增加或者删除
"""

# clickjacking攻击
- 又称作点击劫持攻击，是一种在网页中将恶意代码等隐藏在看似无害的内容之下（如按钮），并诱惑用户点击的手段

# clickjacking攻击场景
## 场景一：
- 如用户收到一封包含视频的电子邮件，但其中的播放按钮并不会播放真正的视频，而是链接进入购物网站，这样当用户视图播放视频时，实际上是被诱骗进入了一个购物网站
## 场景二：
- 用户进入到一个网页中，里面包含了一个非常有诱惑力的按钮A，但是这个按钮上面浮了一个透明的iframe标签，这个iframe标签加载了另外一个网页，并且他将这个网页的某个按钮和原网页中的按钮A重合，所以你在点击按钮A的时候，实际上点的是通过iframe加载的另外一个网页的按钮，

# clickjacking防御
- 像以上场景1，是没有办法避免的，受伤害的是用户。而场景2受伤害的也许是服务器和用户。这种场景是可以避免的，只要设置百度贴吧不允许使用iframe被被加载到其他的页面中，就可以避免这种行为了，我们可以通过在响应头中设置X-Frame-Options来设置这种操作，X-Frame-Options可以设置以下三个值：
    - 1.DENY:不让任何网页使用iframe加载我这个页面
    - 2.SAMEORIGIN:只允许在相同域名下（也就是自己的网站）下使用iframe加载我这个页面
    - 3.ALLOW_FROM_origin:允许任何网页通过iframe来加载这个网页
- 在django中，使用中间件django.middleware.clickjacking.XFrameOptionsMiddleware可以帮我们解决这个漏洞，这个中间件的设置了X-Frame-Option为SAMEORIGIN，也就是只有在自己的网站下才可以使用iframe加载这个网页，这样就可以避免其他别有心机的网页去通过iframe去加载了

# SQL注入
- 所谓SQL注入，就是通过SQL命令插入到表单中或页面请求的查询字符串中，最终达到欺骗服务器执行恶意的SQL命令，具体来说，它是利用现有应用程序，将（恶意的）SQL命令注入到后台数据库引擎的执行能力，它可以通过在Web表单中输入（恶意）SQL语句得到一个存在安全漏洞的网站上的数据库，而不是按照设计者意图去执行SQL语句，比如先前的很多影视网站泄露VIP会员密码大多就是通过WEB表单递交查询字符爆出的的

# SQL防御
- 如何防御SQL的注入，可以从以下几点：
    - 1.永远不要信任用户的的输入，对用户的输入进行校验，可以通过正则表达式，或限制长度，对单引号和双引号进行转换等
    - 2.永远不要使用动态拼装SQL，可以使用参数化的sql语句或者直接使用存储过程进行数据查询存取
    - 3.永远不要使用管理员权限的数据连接，为每个应用使用单独的权限有限的数据库连接
    - 4.不要把机密信息直接存放，加密或者hash掉密码和敏感的信息
    - 5.应用的异常信息应该给出尽可能少的提示，最好使用自定义的错误信息对称原始错误信息进行包装

# 在django中如何防御SQL注入
- 1.使用ORM来做数据库的增删查改，因为ORM使用的是参数化的形式执行sql语句
- 2.如果万一要执行原始SQL语句，那么建议不要拼接url，而是使用参数化的形式

# 所有代码案例都在django_xss中
